In [10]:
%load_ext autoreload
%autoreload 2


In [11]:
from smooth.adam_general.core.adam import ADAM

import numpy as np
import pandas as pd

In [12]:
from typing import List, Union, Dict, Any
from smooth.adam_general._adam_general import adam_fitter, adam_forecaster
from numpy.linalg import eigvals
import nlopt

In [13]:
#### Global level and ETS(A,N,N) ####
np.random.seed(33)  # For reproducibility
n_points = 100
time_series = np.random.normal(100, 10, n_points)  # Random errors
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [14]:
ts_df

,value
2023-01-31,96.811465
2023-02-28,83.970194
2023-03-31,84.647821
2023-04-30,94.295991
2023-05-31,97.832717
...,...
2030-12-31,101.952350
2031-01-31,105.358388
2031-02-28,106.842993
2031-03-31,128.212793


In [15]:
np.random.seed(65)  # For reproducibility
n_points = 39  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)

<positron-console-cell-15>:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.


In [16]:
import time

start_time = time.time()

model = "MAM"
lags = [12]

np.random.seed(33)  # For reproducibility
n_points = 39  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)

adam = ADAM(model, lags)
adam.fit(ts_df)
#print(adam.adam_created)
fc = adam.predict(h = 12)
#print(adam.adam_created)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 0.1255 seconds


,mean,lower_0.025,upper_0.975
2026-04-30,2109.082835,486.581997,4898.817666
2026-05-31,1989.849807,324.695700,5133.446980
2026-06-30,2049.461157,232.317132,5799.444529
2026-07-31,2023.888099,155.399395,6226.741692
2026-08-31,2005.349177,101.094171,6663.759162
2026-09-30,1965.488473,62.723084,7017.814415
2026-10-31,2036.503884,39.468776,7780.188119
2026-11-30,2009.871989,22.586980,8186.916890
2026-12-31,1977.082264,12.243090,8560.898414
2027-01-31,1527.940219,4.930981,7014.641398


In [17]:
import time

start_time = time.time()

model = "MAM"
lags = [12]

np.random.seed(33)  # For reproducibility
n_points = 39  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)

adam = ADAM(model, lags)
adam.fit(ts_df)
#print(adam.adam_created)
fc = adam.predict(h = 12)
#print(adam.adam_created)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 0.1220 seconds


,mean,lower_0.025,upper_0.975
2026-04-30,2109.082835,486.581997,4898.817666
2026-05-31,1989.849807,324.695700,5133.446980
2026-06-30,2049.461157,232.317132,5799.444529
2026-07-31,2023.888099,155.399395,6226.741692
2026-08-31,2005.349177,101.094171,6663.759162
2026-09-30,1965.488473,62.723084,7017.814415
2026-10-31,2036.503884,39.468776,7780.188119
2026-11-30,2009.871989,22.586980,8186.916890
2026-12-31,1977.082264,12.243090,8560.898414
2027-01-31,1527.940219,4.930981,7014.641398


In [43]:
# Load the AirPassengers dataset from Python
import pandas as pd
import numpy as np
# Create the AirPassengers dataset manually
data = [
    112, 118, 132, 129, 121, 135, 148, 148, 136, 119, 104, 118,
    115, 126, 141, 135, 125, 149, 170, 170, 158, 133, 114, 140,
    145, 150, 178, 163, 172, 178, 199, 199, 184, 162, 146, 166,
    171, 180, 193, 181, 183, 218, 230, 242, 209, 191, 172, 194,
    196, 196, 236, 235, 229, 243, 264, 272, 237, 211, 180, 201,
    204, 188, 235, 227, 234, 264, 302, 293, 259, 229, 203, 229,
    242, 233, 267, 269, 270, 315, 364, 347, 312, 274, 237, 278,
    284, 277, 317, 313, 318, 374, 413, 405, 355, 306, 271, 306,
    315, 301, 356, 348, 355, 422, 465, 467, 404, 347, 305, 336,
    340, 318, 362, 348, 363, 435, 491, 505, 404, 359, 310, 337,
    360, 342, 406, 396, 420, 472, 548, 559, 463, 407, 362, 405,
    417, 391, 419, 461, 472, 535, 622, 606, 508, 461, 390, 432
]

# Create a proper datetime index
dates = pd.date_range(start='1949-01-01', periods=len(data), freq='MS')

# Create a pandas Series with the data
air_passengers_series = pd.Series(data, index=dates, name='AirPassengers')

# Create a DataFrame with the time series
ts_df = pd.DataFrame({'value': air_passengers_series})

print("AirPassengers dataset loaded:")
print(ts_df.head())



AirPassengers dataset loaded:
            value
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121


In [19]:
# problem is the selector!!
# lets fix the seelctor!!

In [44]:
import time

start_time = time.time()

model = "ZXZ"
lags = [12]


adam = ADAM(model, lags)
adam.fit(ts_df)
fc = adam.predict(h = 12)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 1.7396 seconds


,mean,lower_0.025,upper_0.975
1961-01-01,447.086033,413.039969,482.461197
1961-02-01,422.186458,387.796375,458.016651
1961-03-01,477.792702,436.502343,520.922290
1961-04-01,500.137506,454.577942,547.841595
1961-05-01,513.528322,464.473323,565.010460
1961-06-01,586.157939,527.690916,647.652474
1961-07-01,674.227055,604.253912,747.978372
1961-08-01,664.643067,593.090022,740.212217
1961-09-01,551.218218,489.821099,616.187988
1961-10-01,488.292391,432.147794,547.816134


In [ ]:
import time

start_time = time.time()

model = "ZXZ"
lags = [12]


adam = ADAM(model, lags)
adam.fit(ts_df)
fc = adam.predict(h = 12)
execution_time = time.time() - start_time
print(f"Execution time: {execution_time:.4f} seconds")
fc

Execution time: 0.1487 seconds


,mean,lower_0.025,upper_0.975
1961-01-01,451.350598,425.396948,477.304248
1961-02-01,427.331519,400.460514,454.202524
1961-03-01,465.526390,437.764325,493.288455
1961-04-01,500.338866,471.709572,528.968160
1961-05-01,508.889261,479.414457,538.364066
1961-06-01,567.881710,537.581288,598.182132
1961-07-01,647.700529,616.592788,678.808269
1961-08-01,632.398319,600.500164,664.296475
1961-09-01,534.226616,501.553713,566.899519
1961-10-01,484.560701,451.127622,517.993781


In [60]:
adam = ADAM(model="ANN", lags=[12])
adam.fit(ts_df)

# print(adam.adam_estimated["B"])

preds = adam.predict(h = 12)
preds

,mean,lower_0.025,upper_0.975
1961-01-01,432.0,365.696577,498.303423
1961-02-01,432.0,338.232800,525.767200
1961-03-01,432.0,317.159103,546.840897
1961-04-01,432.0,299.393154,564.606846
1961-05-01,432.0,283.741039,580.258961
1961-06-01,432.0,269.590445,594.409555
1961-07-01,432.0,256.577632,607.422368
1961-08-01,432.0,244.465600,619.534400
1961-09-01,432.0,233.089731,630.910269
1961-10-01,432.0,222.330167,641.669833


In [23]:
adam = ADAM(model="ANN", lags=[12])
adam.fit(ts_df)

preds = adam.predict(h = 12)
preds

,mean,lower_0.025,upper_0.975
1961-01-01,432.0,365.461874,498.538126
1961-02-01,432.0,432.000000,432.000000
1961-03-01,432.0,432.000000,432.000000
1961-04-01,432.0,432.000000,432.000000
1961-05-01,432.0,432.000000,432.000000
1961-06-01,432.0,432.000000,432.000000
1961-07-01,432.0,432.000000,432.000000
1961-08-01,432.0,432.000000,432.000000
1961-09-01,432.0,432.000000,432.000000
1961-10-01,432.0,432.000000,432.000000


In [24]:
#### Local level and ETS(A,N,N) ####
np.random.seed(41)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [33]:
# ETS(A,N,N) with alpha=0.1
model = ADAM(model="ANN", lags=[12], persistence=0.1)
model.fit(ts_df)
print("ETS(A,N,N) with alpha=0.1:")
print(model.predict(h = 12))

# ETS(A,N,N) with estimated alpha 
model = ADAM(model="ANN", lags=[12])
model.fit(ts_df)
print("\nETS(A,N,N) with estimated parameters:")
print(model.adam_estimated["B"])
print(model.predict(h = 12))

# ETS(M,N,N) with estimated alpha
model = ADAM(model="MNN", lags=[12], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,N,N) with estimated parameters:")
print(model.predict(h = 12))


ETS(A,N,N) with alpha=0.1:
                  mean  lower_0.025  upper_0.975
2033-01-31  174.856443   101.273215   248.439671
2033-02-28  174.856443   174.856443   174.856443
2033-03-31  174.856443   174.856443   174.856443
2033-04-30  174.856443   174.856443   174.856443
2033-05-31  174.856443   174.856443   174.856443
2033-06-30  174.856443   174.856443   174.856443
2033-07-31  174.856443   174.856443   174.856443
2033-08-31  174.856443   174.856443   174.856443
2033-09-30  174.856443   174.856443   174.856443
2033-10-31  174.856443   174.856443   174.856443
2033-11-30  174.856443   174.856443   174.856443
2033-12-31  174.856443   174.856443   174.856443

ETS(A,N,N) with estimated parameters:
[2.81159678e-02 1.86602396e+02]
                  mean  lower_0.025  upper_0.975
2033-01-31  174.669267   102.015067   247.323468
2033-02-28  174.669267   174.669267   174.669267
2033-03-31  174.669267   174.669267   174.669267
2033-04-30  174.669267   174.669267   174.669267
2033-05-31  174.6692

In [26]:
#### Local trend and ETS(A,A,N) ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))


In [27]:
# ETS(A,A,N) with fixed parameters
model = ADAM(model="AAN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df)
print("ETS(A,A,N):")
print(model.predict(h=12))

# ETS(A,Ad,N) with estimated parameters
model = ADAM(model="AAdN", lags=[12])
model.fit(ts_df)
print("\nETS(A,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(A,Ad,N) with fixed parameters
model = ADAM(model="AAdN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df)
print("\nETS(A,Ad,N):")
print(model.predict(h=12))

ETS(A,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.255545   145.244457   185.266633
2033-02-28  165.763972   165.763972   165.763972
2033-03-31  166.272398   166.272398   166.272398
2033-04-30  166.780825   166.780825   166.780825
2033-05-31  167.289251   167.289251   167.289251
2033-06-30  167.797677   167.797677   167.797677
2033-07-31  168.306104   168.306104   168.306104
2033-08-31  168.814530   168.814530   168.814530
2033-09-30  169.322956   169.322956   169.322956
2033-10-31  169.831383   169.831383   169.831383
2033-11-30  170.339809   170.339809   170.339809
2033-12-31  170.848236   170.848236   170.848236

ETS(A,Ad,N) estimated parameters:
[1.27453354e-01 5.47801625e-02 8.07886356e-01 1.01909287e+02
 1.52780264e+00]

ETS(A,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.234972   143.516800   182.953143
2033-02-28  163.700595   163.700595   163.700595
2033-03-31  164.140013   164.140013   164.140013
2033-04-30  164.554699   164

In [28]:
# ETS(M,A,N) with fixed parameters
model = ADAM(model="MAN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("ETS(M,A,N):")
print(model.predict(h=12)['mean'].values)
# ETS(M,Ad,N) with estimated parameters
model = ADAM(model="MAdN", lags=[12], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Ad,N) with fixed parameters
model = ADAM(model="MAdN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Ad,N):")
print(model.predict(h=12)['mean'].values)

ETS(M,A,N):
[165.25624622 165.76453841 166.27283059 166.78112277 167.28941495
 167.79770714 168.30599932 168.8142915  169.32258368 169.83087587
 170.33916805 170.84746023]

ETS(M,Ad,N) estimated parameters:
[  0.20027606   0.           0.99999978 100.10623013   0.486776  ]

ETS(M,Ad,N):
[163.19581608 163.6601315  164.09749108 164.50945975 164.89751158
 165.26303505 165.60733805 165.93165254 166.23713895 166.52489034
 166.79593632 167.05124672]


In [29]:
# ETS(M,A,N) with fixed parameters
model = ADAM(model="MAN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("ETS(M,A,N):")
print(model.predict(h=12))

# ETS(M,Ad,N) with estimated parameters
model = ADAM(model="MAdN", lags=[12], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Ad,N) with fixed parameters
model = ADAM(model="MAdN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Ad,N):")
print(model.predict(h=12))

ETS(M,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.256246   137.193646   193.318846
2033-02-28  165.764538   165.764538   165.764538
2033-03-31  166.272831   166.272831   166.272831
2033-04-30  166.781123   166.781123   166.781123
2033-05-31  167.289415   167.289415   167.289415
2033-06-30  167.797707   167.797707   167.797707
2033-07-31  168.305999   168.305999   168.305999
2033-08-31  168.814292   168.814292   168.814292
2033-09-30  169.322584   169.322584   169.322584
2033-10-31  169.830876   169.830876   169.830876
2033-11-30  170.339168   170.339168   170.339168
2033-12-31  170.847460   170.847460   170.847460

ETS(M,Ad,N) estimated parameters:
[  0.20027606   0.           0.99999978 100.10623013   0.486776  ]

ETS(M,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.195816   135.643407   190.748225
2033-02-28  163.660131   163.660131   163.660131
2033-03-31  164.097491   164.097491   164.097491
2033-04-30  164.509460   164.509460   1

In [30]:
# ETS(M,M,N) with fixed parameters
model = ADAM(model="MMN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("ETS(M,M,N):")
print(model.adam_estimated["B"])
print(model.predict(h=12))

# ETS(M,Md,N) with estimated parameters
model = ADAM(model="MMdN", lags=[12], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Md,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Md,N) with fixed parameters
model = ADAM(model="MMdN", lags=[12], persistence=[0.1, 0.02], distribution="dnorm")
model.fit(ts_df)
print("\nETS(M,Md,N):")
print(model.predict(h=12))


ETS(M,M,N):
[107.1870167    0.99327599]
                  mean  lower_0.025  upper_0.975
2033-01-31  165.593067   137.459503   193.726632
2033-02-28  166.074014   166.074014   166.074014
2033-03-31  166.556358   166.556358   166.556358
2033-04-30  167.040102   167.040102   167.040102
2033-05-31  167.525252   167.525252   167.525252
2033-06-30  168.011810   168.011810   168.011810
2033-07-31  168.499782   168.499782   168.499782
2033-08-31  168.989171   168.989171   168.989171
2033-09-30  169.479981   169.479981   169.479981
2033-10-31  169.972217   169.972217   169.972217
2033-11-30  170.465883   170.465883   170.465883
2033-12-31  170.960982   170.960982   170.960982

ETS(M,Md,N) estimated parameters:
[1.49664624e-01 1.27227893e-02 9.65454489e-01 1.05721451e+02
 9.95168715e-01]

ETS(M,Md,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.174822   135.701216   190.648428
2033-02-28  163.629525   163.629525   163.629525
2033-03-31  164.056822   164.056822   164.056822


/home/config/R/Projects/Packages/smooth/python/src/smooth/adam_general/core/forecaster.py:367: UserWarning: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful.
  warnings.warn("Your model has a potentially explosive multiplicative trend. "


In [31]:
#### Seasonal data ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))  # Random seasonal pattern
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)  # Normalize to sum to 1
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))


In [ ]:
# ETS(A,A,A)
model = ADAM(model="AAA", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("ETS(A,A,A):")
print(model.adam_estimated["B"])
print(model.predict())

# ETS(A,Ad,A) estimated
model = ADAM(model="AAdA", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(A,Ad,A):")
print(model.predict())

NameError: name 'Adam' is not defined

In [ ]:
# ETS(M,A,M)
model = Adam(model="MAM", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("ETS(M,A,M):")
print(model.adam_estimated["B"])
print(model.predict())

# ETS(M,Ad,M) estimated
model = Adam(model="MAdM", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,Ad,M):")
print(model.predict())

ETS(M,A,M):
[1.48159059e-01 1.42680375e-02 7.38461996e-04 2.03782026e+02
 6.27206481e-01 8.65002472e-01 9.46165077e-01 1.03216495e+00
 1.20151786e+00 7.70686649e-01 9.88553426e-01 9.26662454e-01
 9.07359777e-01 1.41730701e+00 1.06717115e+00 8.23421502e-01]
                  mean  lower_0.025  upper_0.975
2033-01-31  146.548717          NaN          NaN
2033-02-28  160.512794          NaN          NaN
2033-03-31  175.332969          NaN          NaN
2033-04-30  204.390755          NaN          NaN
2033-05-31  131.275426          NaN          NaN
2033-06-30  168.607358          NaN          NaN
2033-07-31  158.269810          NaN          NaN
2033-08-31  155.166278          NaN          NaN
2033-09-30  242.711017          NaN          NaN
2033-10-31  183.001773          NaN          NaN
2033-11-30  141.387055          NaN          NaN
2033-12-31  212.314890          NaN          NaN

ETS(M,Ad,M):
                  mean  lower_0.025  upper_0.975
2033-01-31  145.291814          NaN        

In [ ]:
# ETS(M,M,M)
model = Adam(model="MMM", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("ETS(M,M,M):")
print(model.adam_estimated["B"])
print(model.predict())

# ETS(M,Md,M) estimated
model = Adam(model="MMdM", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,Md,M):")
print(model.predict())

CFValue is NaN
ETS(M,M,M):
[1.08756051e-01 2.28520223e-02 3.92570336e-04 2.72423521e+02
 9.80771720e-01 8.56807694e-01 9.44254442e-01 1.02928181e+00
 1.19319890e+00 7.72442704e-01 9.91074074e-01 9.30470656e-01
 9.15833157e-01 1.43033305e+00 1.06086321e+00 8.26935812e-01]
                  mean  lower_0.025  upper_0.975
2033-01-31  146.220025          NaN          NaN
2033-02-28  161.795754          NaN          NaN
2033-03-31  177.084819          NaN          NaN
2033-04-30  206.137724          NaN          NaN
2033-05-31  133.986189          NaN          NaN
2033-06-30  172.610673          NaN          NaN
2033-07-31  162.721959          NaN          NaN
2033-08-31  160.805719          NaN          NaN
2033-09-30  252.180200          NaN          NaN
2033-10-31  187.820320          NaN          NaN
2033-11-30  146.993201          NaN          NaN
2033-12-31  219.443140          NaN          NaN

ETS(M,Md,M):
                  mean  lower_0.025  upper_0.975
2033-01-31  144.821477      

/home/config/Cursor/smooth/.venv/lib/python3.12/site-packages/smooth/adam_general/core/forecaster.py:134: UserWarning: Your model has a potentially explosive multiplicative trend. I cannot do anything about it, so please just be careful.
  warnings.warn("Your model has a potentially explosive multiplicative trend. "
